In [3]:
# Import packages
import os
from zipfile import ZipFile
import boto3

In [4]:
# Put credentials file in .aws directory; check if file is found
# https://docs.aws.amazon.com/credref/latest/refdocs/file-location.html
# %USERPROFILE%\.aws\credentials

os.listdir(os.environ['USERPROFILE'] + '\.aws\\')

['credentials']

In [13]:
# Zip the .py file that contains the lambda function
ZipFile('reference/deploy1.zip', mode='w').write('reference/lambda_function1.py')
ZipFile('reference/deploy2.zip', mode='w').write('reference/lambda_function2.py')

In [14]:
awsLambda = boto3.client('lambda', 'us-east-1')

# Create a Lambda function #1
response1 = awsLambda.create_function(
    FunctionName='weisman-lambda1-pass-data',
    Runtime='python3.8',
    Role='arn:aws:iam::475434563362:role/service-role/weisman-test-role-03ak8xfv', # copied from IAM
    Handler='string',
    Code={
      'ZipFile': open('reference/deploy1.zip', 'rb').read()
    },
    Description='Step 1 of 2 in Step Function to pass data',
    Timeout=123,
    Publish=False)

# Create a Lambda function #1
response2 = awsLambda.create_function(
    FunctionName='weisman-lambda2-pass-data',
    Runtime='python3.8',
    Role='arn:aws:iam::475434563362:role/service-role/weisman-test-role-03ak8xfv', # copied from IAM
    Handler='string',
    Code={
      'ZipFile': open('reference/deploy2.zip', 'rb').read()
    },
    Description='Step 2 of 2 in Step Function to pass data',
    Timeout=123,
    Publish=False)

print(response1, response2)

{'ResponseMetadata': {'RequestId': '12fa8d4b-cd26-4ebf-8a93-659ef937b9b0', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 24 Feb 2021 23:15:42 GMT', 'content-type': 'application/json', 'content-length': '963', 'connection': 'keep-alive', 'x-amzn-requestid': '12fa8d4b-cd26-4ebf-8a93-659ef937b9b0'}, 'RetryAttempts': 0}, 'FunctionName': 'weisman-lambda1-pass-data', 'FunctionArn': 'arn:aws:lambda:us-east-1:475434563362:function:weisman-lambda1-pass-data', 'Runtime': 'python3.8', 'Role': 'arn:aws:iam::475434563362:role/service-role/weisman-test-role-03ak8xfv', 'Handler': 'string', 'CodeSize': 433, 'Description': 'Step 1 of 2 in Step Function to pass data', 'Timeout': 123, 'MemorySize': 128, 'LastModified': '2021-02-24T23:15:42.770+0000', 'CodeSha256': '52Jc4mnlT8dDZWmT5r2PmCuG9UV+DEl38zqhWPgUM5Y=', 'Version': '$LATEST', 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': '69110def-00bb-4aec-984e-afae537551e7', 'State': 'Active', 'LastUpdateStatus': 'Successful', 'PackageType': 'Zi

In [10]:
# Create Step Function
sfn = boto3.client('stepfunctions', 'us-east-1')

In [20]:
# Create definition using Amazon States Language 
definition = """{
  "Comment": "My test to prove passing parameters between Lambdas",
  "StartAt": "weisman-lambda1-pass-data",
  "States": {
    "weisman-lambda1-pass-data": {
      "Type": "Task",
      "Resource": "arn:aws:lambda:us-east-1:475434563362:function:weisman-lambda1-pass-data",
      "ResultPath": "$",
      "Next": "weisman-lambda2-pass-data"
    },
      "weisman-lambda2-pass-data": {
      "Type": "Task",
      "Resource": "arn:aws:lambda:us-east-1:475434563362:function:weisman-lambda2-pass-data",
      "End": true
    }
  }
}"""

In [22]:
# Create Step Function
response = sfn.create_state_machine(
    name='weisman-test-created-from-boto3',
    definition=definition,
    roleArn='arn:aws:iam::475434563362:role/service-role/StepFunctions-Helloworld-role-1b31bef2', # copied from IAM
    type='STANDARD')

response

{'stateMachineArn': 'arn:aws:states:us-east-1:475434563362:stateMachine:weisman-test-created-from-boto3',
 'creationDate': datetime.datetime(2021, 2, 24, 18, 23, 48, 706000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'c0b2edb5-c019-4fc0-81ce-42f63f314ae4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c0b2edb5-c019-4fc0-81ce-42f63f314ae4',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '136'},
  'RetryAttempts': 0}}